# Instal modul yang digunakan

In [ ]:
# pos_tagging
!pip install python-crfsuite
!pip install sklearn-pycrfsuite

  Using cached sklearn_pycrfsuite-0.4.0-py2.py3-none-any.whl
  Using cached python-crfsuite-extension-0.9.7.tar.gz (485 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for python-crfsuite-extension
  Running setup.py clean for python-crfsuite-extension
Failed to build python-crfsuite-extension
ERROR: Could not build wheels for python-crfsuite-extension, which is required to install pyproject.toml-based projects


# Import data & Cleaning dasar

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/RisetHibahPemerintah/alodokter_topik.csv')

In [ ]:
df.dtypes

topik             object
Title             object
Question          object
Question Date     object
Answer            object
Answer Date       object
By Doctor         object
Year             float64
dtype: object

In [ ]:
duplicates = df[df.duplicated()]
print(duplicates)
df = df.drop_duplicates()

Empty DataFrame
Columns: [topik, Title, Question, Question Date, Answer, Answer Date, By Doctor, Year]
Index: []


In [ ]:
null_rows = df[df.isnull().any(axis=1)]
print(null_rows)
df = df.dropna().reset_index(drop=True)

                         topik  \
3975   gangguan kecemasan umum   
5560   gangguan tidur berjalan   
7010          kesehatan mental   
11713                psikologi   

                                                   Title  \
3975   Cara untuk mengatasi rasa cemas dan takut di m...   
5560   Tidur sambil berjalan disertai mata membuka da...   
7010   Kenapa setiap ada orang yang  lewat depan ruma...   
11713  Kenapa setiap ada orang yang  lewat depan ruma...   

                                                Question  \
3975   Dok . Terkadang ketika malam mau tiba saya suk...   
5560   Halo dok, saya berumur 11 tahun, saya menderit...   
7010   siang dok saya berada di dalam rumah setiap ad...   
11713  siang dok saya berada di dalam rumah setiap ad...   

              Question Date            Answer            Answer Date  \
3975   22 Maret 2018, 23:56  Answer not found  Answer date not found   
5560   23 Maret 2018, 19:56  Answer not found  Answer date not found   
7010    6 M

In [ ]:
df = df.drop(columns=['Title', 'Question', 'Question Date', 'Answer Date', 'By Doctor'])
df['Year'] = df['Year'].astype(int)

In [ ]:
df.head()

,topik,Answer,Year
0,adhd,"Alo, selamat siang Dari penjelasan anda, a...",2024
1,adhd,"Alo, terimakasih atas pertanyaannya. ...",2024
2,adhd,"Alo, terimakasih atas pertanyaannya. ...",2024
3,adhd,"Alo, terimakasih atas pertanyaannya. ...",2024
4,adhd,"Alo, terimakasih atas pertanyaannya. ...",2024


# menghapus karakter spesial

In [ ]:
import re

# Melakukan case folding pada kolom 'Answer'
def remove_tweet_special (text):
  # menghapus angka dari kalimat
  hasil = re.sub(r"\d+", " ", text)

  # menghapus tanda baca dari text
  hasil = re.sub(r'[^\w\s]', ' ', hasil)

  # # menggabungkan karakter tertentu
  # hasil = re.sub(r'\s+k\s+e\s+', ' ke ', hasil)

  # Menghilangkan karakter tunggal
  hasil = re.sub(r"\b[a-zA-Z]\b", " ", hasil)

  # menghapus spasi berlebih di awal dan akhir
  hasil = hasil.strip()

  # Menghapus beberapa spasi menjadi satu spasi
  hasil = re.sub('\s+',' ',hasil)

  return hasil

df['Answer'] = df['Answer'].apply(remove_tweet_special)
df.head()

,topik,Answer,Year
0,adhd,Alo selamat siang Dari penjelasan anda anda me...,2024
1,adhd,Alo terimakasih atas pertanyaannya ADHD sering...,2024
2,adhd,Alo terimakasih atas pertanyaannya Sering mera...,2024
3,adhd,Alo terimakasih atas pertanyaannya Mudah gelis...,2024
4,adhd,Alo terimakasih atas pertanyaannya ADHD attent...,2024


# Casefolding

In [ ]:
# Melakukan case folding pada kolom 'Answer'
def casefolding (text):
  # mengubah kalimat menjadi huruf kecil
  lower_case = text.lower()

  return lower_case

df['Answer'] = df['Answer'].apply(casefolding)
df.head()

,topik,Answer,Year
0,adhd,alo selamat siang dari penjelasan anda anda me...,2024
1,adhd,alo terimakasih atas pertanyaannya adhd sering...,2024
2,adhd,alo terimakasih atas pertanyaannya sering mera...,2024
3,adhd,alo terimakasih atas pertanyaannya mudah gelis...,2024
4,adhd,alo terimakasih atas pertanyaannya adhd attent...,2024


### Intall swifter

In [ ]:
!pip install swifter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16507 sha256=ee22a1c21f301a30d63020b2579b51deebdeb61d0347cd2134f4e5e9444f3f4d
  Stored in directory: /root/.cache/pip/wheels/e4/cf/51/0904952972ee2c7aa3709437065278dc534ec1b8d2ad41b443
Successfully built swifter


# Normalisasi teks

In [ ]:
# Fungsi untuk membaca file normalization.txt dan membuat dictionary
def load_normalization_dict(file_path):
    normalization_dict = {}
    with open(file_path, 'r') as file:
        for line in file:
            key, value = line.strip().split(':')
            normalization_dict[key] = value
    return normalization_dict

# Fungsi untuk mengganti kata-kata dalam teks berdasarkan dictionary
def normalize_text(text, normalization_dict):
    words = text.split()
    normalized_words = []
    for word in words:
        # Ganti kata jika ada dalam kamus normalisasi
        if word in normalization_dict:
            normalized_words.append(normalization_dict[word])
        else:
            normalized_words.append(word)
    return ' '.join(normalized_words)

# Memuat dictionary normalisasi dari file
normalization_dict = load_normalization_dict('/content/drive/MyDrive/Colab Notebooks/RisetHibahPemerintah/normalisasi.txt')

# Normalisasi kolom 'Answer'
df['Normalized_Answer'] = df['Answer'].apply(lambda x: normalize_text(x, normalization_dict))
df['Normalized_Answer'] = df['Normalized_Answer'].apply(remove_tweet_special)

# # Tampilkan DataFrame setelah normalisasi
# print(df)

# Stop word

In [ ]:
# Import library
import nltk
from nltk.corpus import stopwords
import string
import pandas as pd

# Pastikan untuk mengunduh stopwords dari NLTK
nltk.download('stopwords')

# Inisialisasi stop words dan tambahkan tanda baca sebagai stop words tambahan
stop_words = set(stopwords.words('indonesian')) | set(string.punctuation)

# ---------------------------- Baca stopword manual dari file txt  ------------------------------------
def read_manual_stopwords(file_path):
    with open(file_path, 'r') as file:
        manual_stopwords = file.read().splitlines()
    return set(manual_stopwords)

# Path ke file .txt yang berisi daftar kata-kata yang tidak diperlukan
manual_stopwords_file = '/content/drive/MyDrive/Colab Notebooks/RisetHibahPemerintah/manual_stopword.txt'

# Membaca daftar kata-kata yang tidak diperlukan dari file .txt
manual_stopwords = read_manual_stopwords(manual_stopwords_file)

# Tambahkan manual stopwords ke dalam stop_words
stop_words.update(manual_stopwords)

# Fungsi untuk menghapus stop words dari sebuah teks
def remove_stop_words(text):
    # Tokenisasi teks
    words = text.split()
    # Hapus stop words
    filtered_words = [word for word in words if word.lower() not in stop_words]
    # Gabungkan kembali kata-kata yang telah difilter menjadi sebuah string
    return ' '.join(filtered_words)

# Terapkan fungsi remove_stop_words pada kolom 'Answer'
df['Answer_sw'] = df['Normalized_Answer'].apply(remove_stop_words)
df['Answer_sw'] = df['Answer_sw'].apply(remove_tweet_special)

# Tampilkan DataFrame yang telah diubah
df[['Normalized_Answer','Answer_sw']].head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,Normalized_Answer,Answer_sw
0,alo selamat siang dari penjelasan anda anda me...,penjelasan susah fokus susah fokus menandakan ...
1,alo terimakasih atas pertanyaannya adhd sering...,pertanyaannya adhd gejalanya kisaran usia geja...
2,alo terimakasih atas pertanyaannya sering mera...,pertanyaannya cemas gugup tegang sulit konsent...
3,alo terimakasih atas pertanyaannya mudah gelis...,pertanyaannya mudah gelisah rungsing sulit fok...
4,alo terimakasih atas pertanyaannya adhd attent...,pertanyaannya adhd attention deficit hyperacti...


# Tokenization

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

# NLTK word tokenize
def word_tokenize_wrapper(text):
  return word_tokenize(text)

df['Answer_tkn'] = df['Answer_sw'].apply(word_tokenize_wrapper)
df[['Answer_sw','Answer_tkn']].head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


,Answer_sw,Answer_tkn
0,penjelasan susah fokus susah fokus menandakan ...,"[penjelasan, susah, fokus, susah, fokus, menan..."
1,pertanyaannya adhd gejalanya kisaran usia geja...,"[pertanyaannya, adhd, gejalanya, kisaran, usia..."
2,pertanyaannya cemas gugup tegang sulit konsent...,"[pertanyaannya, cemas, gugup, tegang, sulit, k..."
3,pertanyaannya mudah gelisah rungsing sulit fok...,"[pertanyaannya, mudah, gelisah, rungsing, suli..."
4,pertanyaannya adhd attention deficit hyperacti...,"[pertanyaannya, adhd, attention, deficit, hype..."


# Bigram

In [ ]:
import nltk
from nltk import bigrams
from collections import Counter

# Pastikan nltk sudah mendownload data yang diperlukan
nltk.download('punkt')

# Membuat kolom baru untuk bigram
df['Bigrams'] = df['Answer_tkn'].apply(lambda x: list(bigrams(x)))

# Menggabungkan semua bigram ke dalam satu list
all_bigrams = [bigram for sublist in df['Bigrams'] for bigram in sublist]

# Menghitung frekuensi bigram
bigram_freq = Counter(all_bigrams)

# Menampilkan bigram yang paling umum
most_common_bigrams = bigram_freq.most_common(100)
most_common_bigrams

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[(('dokter', 'spesialis'), 4783),
 (('gangguan', 'kecemasan'), 4447),
 (('gangguan', 'mental'), 4369),
 (('obat', 'obatan'), 3931),
 (('gangguan', 'kejiwaan'), 3760),
 (('gangguan', 'kepribadian'), 3744),
 (('depresi', 'gangguan'), 3265),
 (('dokter', 'psikiater'), 3123),
 (('langsung', 'dokter'), 2855),
 (('gangguan', 'bipolar'), 2836),
 (('mengalami', 'gangguan'), 2465),
 (('hidup', 'sehat'), 2422),
 (('suasana', 'hati'), 2422),
 (('berkonsultasi', 'dokter'), 2367),
 (('aktivitas', 'sehari'), 2337),
 (('gangguan', 'cemas'), 2334),
 (('psikolog', 'psikiater'), 2301),
 (('keluhan', 'alami'), 2143),
 (('kesehatan', 'mental'), 2143),
 (('gaya', 'hidup'), 2033),
 (('kondisi', 'alami'), 1920),
 (('pemeriksaan', 'fisik'), 1918),
 (('pasca', 'trauma'), 1777),
 (('spesialis', 'kejiwaan'), 1754),
 (('serangan', 'panik'), 1716),
 (('mengganggu', 'aktivitas'), 1713),
 (('informasi', 'membantu'), 1678),
 (('gejala', 'gejala'), 1656),
 (('gangguan', 'stres'), 1650),
 (('gangguan', 'kesehatan'), 15

In [ ]:
df[['Answer_tkn','Bigrams']].head()

,Answer_tkn,Bigrams
0,"[penjelasan, susah, fokus, susah, fokus, menan...","[(penjelasan, susah), (susah, fokus), (fokus, ..."
1,"[pertanyaannya, adhd, gejalanya, kisaran, usia...","[(pertanyaannya, adhd), (adhd, gejalanya), (ge..."
2,"[pertanyaannya, cemas, gugup, tegang, sulit, k...","[(pertanyaannya, cemas), (cemas, gugup), (gugu..."
3,"[pertanyaannya, mudah, gelisah, rungsing, suli...","[(pertanyaannya, mudah), (mudah, gelisah), (ge..."
4,"[pertanyaannya, adhd, attention, deficit, hype...","[(pertanyaannya, adhd), (adhd, attention), (at..."


# Lemmatization

In [ ]:
# !pip install Sastrawi
!pip install PySastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 kB 4.3 MB/s eta 0:00:00


In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter
import pandas as pd

# Inisialisasi stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Fungsi untuk stemming
def stemmed_wrapper(term):
    return stemmer.stem(term)

# Membuat kamus term
term_dict = {}

# Membentuk set dari semua term untuk menghindari duplikasi dan mengurangi jumlah iterasi
unique_terms = set()
for doc in df['Answer_tkn']:
    for term in doc:
        unique_terms.add(term)

# Melakukan stemming dengan swifter dan mengupdate term_dict
unique_terms = list(unique_terms)  # Konversi kembali ke list agar bisa digunakan dengan swifter
df_terms = pd.DataFrame(unique_terms, columns=['term'])
df_terms['stemmed'] = df_terms['term'].swifter.apply(stemmed_wrapper)

# Konversi kembali ke dictionary
term_dict = pd.Series(df_terms.stemmed.values, index=df_terms.term).to_dict()

# # Menampilkan hasil
# for term, stemmed in term_dict.items():
#     print(term, ":", stemmed)
#     print(len(term_dict))
#     print("------------------------")

Pandas Apply:   0%|          | 0/24723 [00:00<?, ?it/s]

In [ ]:
# Fungsi untuk mendapatkan term yang telah di-stemming
def get_stemmed_term(doc):
    return [term_dict.get(term, term) for term in doc]

# Mengaplikasikan fungsi ke DataFrame
df['Answer_lemma'] = df['Answer_tkn'].apply(get_stemmed_term)

# Menampilkan hasil
df[['Answer_tkn', 'Answer_lemma']].head()

,Answer_tkn,Answer_lemma
0,"[penjelasan, susah, fokus, susah, fokus, menan...","[jelas, susah, fokus, susah, fokus, tanda, ala..."
1,"[pertanyaannya, adhd, gejalanya, kisaran, usia...","[tanya, adhd, gejala, kisar, usia, gejala, adh..."
2,"[pertanyaannya, cemas, gugup, tegang, sulit, k...","[tanya, cemas, gugup, tegang, sulit, konsentra..."
3,"[pertanyaannya, mudah, gelisah, rungsing, suli...","[tanya, mudah, gelisah, rungsing, sulit, fokus..."
4,"[pertanyaannya, adhd, attention, deficit, hype...","[tanya, adhd, attention, deficit, hyperactivit..."


### Cleaning hasil lemma

In [ ]:
# Buat kolom baru 'Answer_untoken' yang berisi teks yang telah diuntoken
df['Answer_lemma_cln'] = df['Answer_lemma'].apply(lambda tokens: ' '.join(tokens))

# normalisasi
df['Answer_lemma_cln'] = df['Answer_lemma_cln'].apply(lambda x: normalize_text(x, normalization_dict))
df['Answer_lemma_cln'] = df['Answer_lemma_cln'].apply(remove_tweet_special)

# stopword
df['Answer_lemma_cln'] = df['Answer_lemma_cln'].apply(remove_stop_words)
df['Answer_lemma_cln'] = df['Answer_lemma_cln'].apply(remove_tweet_special)

df['Answer_lemma_cln'] = df['Answer_lemma_cln'].apply(word_tokenize_wrapper)

In [ ]:
df[['topik', 'Answer_lemma_cln']].head()

,topik,Answer_lemma_cln
0,adhd,"[susah, fokus, susah, fokus, tanda, alami, adh..."
1,adhd,"[adhd, gejala, kisar, usia, gejala, adhd, taha..."
2,adhd,"[cemas, gugup, tegang, sulit, konsentrasi, mud..."
3,adhd,"[mudah, gelisah, rungsing, sulit, fokus, tanda..."
4,adhd,"[adhd, attention, deficit, hyperactivity, diso..."


### menghitung frekuensi kata

In [ ]:
from collections import Counter
# Menghitung frekuensi kata-kata dalam kolom 'Answer_lemma_cln'
word_freq = Counter()
for tokens in df['Answer_lemma_cln']:
    word_freq.update(tokens)

# Konversi hasil Counter ke DataFrame untuk ditampilkan
word_freq_df = pd.DataFrame(list(word_freq.items()), columns=['Kata', 'Frekuensi'])

# Menampilkan hasil berdasarkan frekuensi tertinggi
word_freq_df.sort_values(by='Frekuensi', ascending=False, inplace=True)
word_freq_df

,Kata,Frekuensi
7,ganggu,57049
40,kondisi,28632
3,alami,26042
41,dokter,26030
100,keluh,22173
...,...,...
11602,jakob,1
11601,creutzfeldt,1
11600,therew,1
11598,mengkonsumi,1


In [ ]:
# Filter kata-kata yang frekuensinya > 1500
filtered_words_df = word_freq_df[word_freq_df['Frekuensi'] > 1500]
filtered_words_df.head(10)

,Kata,Frekuensi
7,ganggu,57049
40,kondisi,28632
3,alami,26042
41,dokter,26030
100,keluh,22173
43,periksa,19126
82,obat,18567
38,bantu,16730
90,cemas,16549
46,gejala,15172


### melihat kata-kata yang tidak umum muncul secara keseluruhan row data

In [ ]:
# Menghitung frekuensi munculnya setiap kata di seluruh kolom 'Answer_lemma_cln'
all_tokens = [token for tokens in df['Answer_lemma_cln'] for token in tokens]
word_counts = Counter(all_tokens)

# Menyaring kata-kata yang frekuensinya kurang dari 5
rare_words = [word for word, count in word_counts.items() if count < 3]

In [ ]:
rare_words

['rungsing',
 'edukatif',
 'kehialng',
 'sepertihitung',
 'danmriotak',
 'seesorang',
 'sial',
 'terutam',
 'contognya',
 'diperhintakan',
 'mendapingi',
 'elo',
 'intruksi',
 'mengahabiskan',
 'meningaktkan',
 'logiknya',
 'deficithyperactivity',
 'inklusif',
 'dificit',
 'rash',
 'frozen',
 'talkshow',
 'maurha',
 'note',
 'kegiatn',
 'bsdtalia',
 'anakgejala',
 'hiperaktifdan',
 'huriyatul',
 'honi',
 'adh',
 'penananganan',
 'ajaka',
 'haryati',
 'priotitas',
 'attentian',
 'aran',
 'keponaan',
 'cd',
 'tintin',
 'aritmatiknya',
 'pos',
 'tutor',
 'kalkulator',
 'nailun',
 'hilly',
 'slcenderung',
 'faldy',
 'developmental',
 'khya',
 'unvoluntary',
 'epidemiologi',
 'elya',
 'mustinya',
 'tuh',
 'ngeh',
 'bagan',
 'utisme',
 'babbling',
 'frase',
 'kelimpungan',
 'kejangggalam',
 'anaksulit',
 'iniberdampak',
 'gelaja',
 'mamass',
 'maslaha',
 'volunter',
 'gait',
 'hyperactivitu',
 'geliat',
 'anisah',
 'mukulkan',
 'furnitur',
 'asar',
 'bebicara',
 'bilingual',
 'definis',
 'pe

In [ ]:
# Menampilkan jumlah kata-kata yang langka
print("Jumlah kata-kata langka (rare words):", len(rare_words))

Jumlah kata-kata langka (rare words): 10319


In [ ]:
sering_muncul = [word for word, count in word_counts.items() if count > 10000]
print("Jumlah kata-kata yang terlalu sering muncul:", len(sering_muncul))

Jumlah kata-kata yang terlalu sering muncul: 20


In [ ]:
sering_muncul

['alami',
 'aktivitas',
 'ganggu',
 'laku',
 'pikir',
 'bantu',
 'konsultasi',
 'kondisi',
 'dokter',
 'periksa',
 'gejala',
 'psikiater',
 'obat',
 'cemas',
 'keluh',
 'hidup',
 'sehat',
 'jiwa',
 'sakit',
 'depresi']

### Cleaning hasil lemma2

In [ ]:
# Konversi rare_words menjadi set untuk efisiensi
rare_words_set = set(rare_words)

# Fungsi untuk melakukan stop word terhadap kolom 'Answer_fix'
def stop_words_rare_word(text, stop_words_set):
    tokens = text.split()
    filtered_tokens = [token for token in tokens if token not in stop_words_set]
    return ' '.join(filtered_tokens)

In [ ]:
# Buat kolom baru 'Answer_untoken' yang berisi teks yang telah diuntoken
df['Answer_fix'] = df['Answer_lemma_cln'].apply(lambda tokens: ' '.join(tokens))

# Terapkan fungsi stop_words_rare_word ke kolom 'Answer_fix'
df['Answer_fix'] = df['Answer_fix'].apply(stop_words_rare_word, stop_words_set=rare_words_set)
df['Answer_fix'] = df['Answer_fix'].apply(remove_tweet_special)

df['Answer_fix'] = df['Answer_fix'].apply(word_tokenize_wrapper)

In [ ]:
from collections import Counter
# Menghitung frekuensi kata-kata dalam kolom 'Answer_fix'
word_freq = Counter()
for tokens in df['Answer_fix']:
    word_freq.update(tokens)

# Konversi hasil Counter ke DataFrame untuk ditampilkan
word_freq_df = pd.DataFrame(list(word_freq.items()), columns=['Kata', 'Frekuensi'])

# Menampilkan hasil berdasarkan frekuensi tertinggi
word_freq_df.sort_values(by='Frekuensi', ascending=False, inplace=True)
print("frekuensi kata dalam dataset sekarang adalah: ",len(word_freq_df))

frekuensi kata dalam dataset sekarang adalah:  5654


In [ ]:
df['Answer_fix'].head()

0    [susah, fokus, susah, fokus, tanda, alami, adh...
1    [adhd, gejala, kisar, usia, gejala, adhd, taha...
2    [cemas, gugup, tegang, sulit, konsentrasi, mud...
3    [mudah, gelisah, sulit, fokus, tanda, alami, a...
4    [adhd, attention, deficit, hyperactivity, diso...
Name: Answer_fix, dtype: object

# POS tagging

In [ ]:
import pycrfsuite
import pandas as pd
from nltk import word_tokenize
from nltk.tag import CRFTagger

# Membuat objek CRFTagger
ct = CRFTagger()

# Menetapkan model yang sudah dilatih
ct.set_model_file('/content/drive/MyDrive/Colab Notebooks/RisetHibahPemerintah/all_indo_man_tag_corpus_model.crf.tagger')

In [ ]:
pos_tagged_lemma = ct.tag_sents(df.Answer_fix)
df['POS'] = pos_tagged_lemma

In [ ]:
df[['Answer_fix','POS']].head()

,Answer_fix,POS
0,"[susah, fokus, susah, fokus, tanda, alami, adh...","[(susah, VB), (fokus, NN), (susah, NN), (fokus..."
1,"[adhd, gejala, kisar, usia, gejala, adhd, taha...","[(adhd, NN), (gejala, NN), (kisar, NN), (usia,..."
2,"[cemas, gugup, tegang, sulit, konsentrasi, mud...","[(cemas, NN), (gugup, NN), (tegang, NN), (suli..."
3,"[mudah, gelisah, sulit, fokus, tanda, alami, a...","[(mudah, JJ), (gelisah, VB), (sulit, JJ), (fok..."
4,"[adhd, attention, deficit, hyperactivity, diso...","[(adhd, FW), (attention, FW), (deficit, FW), (..."


In [ ]:
# Fungsi untuk mencari kata-kata berdasarkan setiap jenis kode tag POS
def find_words_by_pos(df):
    pos_tags = set()
    words_by_pos = {}

    # Iterasi melalui setiap baris data
    for tags in df['POS']:
        for word, pos in tags:
            if pos not in words_by_pos:
                words_by_pos[pos] = set()
            words_by_pos[pos].add(word)
            pos_tags.add(pos)

    return words_by_pos

# Memanggil fungsi untuk mencari kata-kata berdasarkan setiap jenis kode tag POS
words_by_pos = find_words_by_pos(df)

# Menampilkan kata-kata untuk setiap jenis kode tag POS
for pos, words in words_by_pos.items():
    print(f"Kata-kata dengan tag POS '{pos}':")
    print(words)
    print()

Kata-kata dengan tag POS 'VB':
{'serati', 'secar', 'badut', 'rekam', 'raksa', 'klarifikasi', 'candu', 'kabur', 'hasut', 'bersiteru', 'nyeri', 'sugar', 'bimbang', 'majalah', 'ginjal', 'rontok', 'bibi', 'onani', 'uap', 'guncang', 'seni', 'musibah', 'runtuh', 'hawa', 'subur', 'republik', 'sandar', 'psikotropika', 'mari', 'buyar', 'jenuh', 'timbang', 'telpon', 'sederhana', 'kardiomiopati', 'matang', 'airish', 'bajar', 'bakar', 'ngompol', 'kerucut', 'cacat', 'panas', 'keluh', 'sepi', 'dorong', 'lingkup', 'simak', 'selaput', 'begadang', 'kali', 'antuk', 'pisang', 'radang', 'gergaji', 'cerdas', 'pejam', 'obrol', 'kesal', 'aplikasi', 'susah', 'lipat', 'gemuk', 'sakit', 'leceh', 'langgar', 'ganggun', 'tidur', 'trauma', 'ketuk', 'meni', 'sarang', 'bodoh', 'hidung', 'gugup', 'introvert', 'tuang', 'kecam', 'janji', 'rujuk', 'turun', 'jenis', 'rangsang', 'papar', 'dejavu', 'lacak', 'batang', 'olah', 'amuk', 'tolong', 'hias', 'kilas', 'bawa', 'punggung', 'segenap', 'benta', 'affect', 'iring', 'tang'

# Simpan df

In [ ]:
# Simpan DataFrame
df.to_csv('data_preprocessing.csv', index=False)
df.to_excel('data_preprocessing.xlsx', index=False)

In [ ]:
data_bersih = df[['topik','Answer','Year','Answer_fix']].copy()
data_bersih.to_csv('/content/drive/MyDrive/Colab Notebooks/RisetHibahPemerintah/data_fix.csv', index=False)
data_bersih.to_excel('data_fix.xlsx', index=False)